In [1]:
from fastai2.text.all import *

In [2]:
path = Path.home()/'.fastai/data/nlp-getting-started'

In [3]:
df_train = pd.read_csv(path/'train.csv')
df_test = pd.read_csv(path/'test.csv')
df = pd.concat([df_train, df_test], axis=0, sort=True)
df.head(2)

,id,keyword,location,target,text
0,1,NaN,NaN,1.0,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
1,4,NaN,NaN,1.0,Forest fire near La Ronge Sask. Canada


### Training the language model

In [4]:
df_tok, word_counts = tokenize_df(df, text_cols='text')

In [5]:
dls_lm = TextDataLoaders.from_df(df_tok, path, text_col='text',is_lm=True, vocab=make_vocab(word_counts))

In [6]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=Perplexity())
learn.fit_one_cycle(10, 2e-2, moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,perplexity,time
0,3.272980,1.366911,3.923212,00:13
1,2.035723,0.731892,2.079010,00:13
2,1.467342,0.644760,1.905530,00:13
3,1.180095,0.613830,1.847494,00:13
4,1.013120,0.597554,1.817667,00:13
5,0.928538,0.583679,1.792622,00:13
6,0.857455,0.572905,1.773412,00:13
7,0.809753,0.566883,1.762764,00:13
8,0.789257,0.564288,1.758196,00:13
9,0.767127,0.564056,1.757788,00:13


In [20]:
learn.save('lm')

### Using the language model for classification

In [12]:
dls_clas = TextDataLoaders.from_df(df_tok[df_tok.target.notnull()].copy(), path=path, vocab=make_vocab(word_counts), text_col='text',
                                  label_col='target')

In [18]:
cls_learner = text_classifier_learner(dls_clas, AWD_LSTM, path=path, metrics=accuracy)
cls_learner.load('lm.pkl')
cls_learner.fit_one_cycle(3, 2e-2, moms=(0.8, 0.7, 0.8))

FileNotFoundError: [Errno 2] No such file or directory: '/home/achinta/.fastai/data/nlp-getting-started/models/lm.pkl.pth'

In [21]:
cls_learner.model

SequentialRNN(
  (0): SentenceEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(328, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(328, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): LinBnDrop(
        (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Dropout(p=0.2, inplace=False)
        (2): Linear(in_features=1200, out_features=50, bias=False)
        (3): ReLU(inplace=True)
  

In [22]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(440, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(440, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=440, bias=True)
    (output_dp): RNNDropout()
  )
)